In [1]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from tqdm import tqdm_notebook
from itertools import product

import matplotlib.pyplot as plt
import statsmodels.api as sm
import plotly.express as px
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# import data
df = pd.read_csv('rules2023.csv')
df

,Unnamed: 0,antecedents,consequents,support,confidence,lift
0,0,"frozenset({'HARMONY MELON', 'HARMONY LEMON', '...",frozenset({'HARMONY STRAWBERY'}),0.000113,1,1235.260000
1,1,"frozenset({'DELFI TOP TRIP/CHO 9GR', 'MOMOGI J...",frozenset({'MOMOGI CKL 6GR'}),0.000113,1,686.255556
2,2,"frozenset({'INDOMILK KID VNL 115ML', 'CLOUD 9 ...",frozenset({'CLOUD 9 CHOCO 15 4G'}),0.000113,1,571.879630
3,3,"frozenset({'CLOUD 9 VANILA 15 4G', 'INDOMILK K...",frozenset({'CLOUD 9 CHOCO 15 4G'}),0.000146,1,571.879630
4,4,"frozenset({'INDOMILK KID VNL 115ML', 'CLOUD 9 ...",frozenset({'CLOUD 9 CHOCO 15 4G'}),0.000146,1,571.879630
5,5,"frozenset({'EKONOMI E900K', 'NUVO SOAP YLOW 72...",frozenset({'NUVO SOAP BLUE 72G'}),0.000113,1,388.446541
6,6,"frozenset({'INDOMILK STR 190ML BTL', 'INDOMILK...",frozenset({'INDOMILK KID VNL 115ML'}),0.000113,1,348.943503
7,7,"frozenset({'TONGTJI JASMINE 5 S', 'INDOMIE GRG...",frozenset({'INDOMIE AYAM SPECIAL'}),0.000113,1,284.622120
8,8,"frozenset({'KRIPIK SGKG TURBO 180G', 'TWISTER ...",frozenset({'OREO SOFT CAKE 16GR'}),0.000113,1,236.639847
9,9,"frozenset({'INDOMILK KID VNL 115ML', 'TEH GELA...",frozenset({'INDOMILK KID CKL 115ML'}),0.000162,1,183.818452


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   18 non-null     int64  
 1   antecedents  18 non-null     object 
 2   consequents  18 non-null     object 
 3   support      18 non-null     float64
 4   confidence   18 non-null     int64  
 5   lift         18 non-null     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 996.0+ bytes


In [4]:
df = df.head(3)
df

,Unnamed: 0,antecedents,consequents,support,confidence,lift
0,0,"frozenset({'HARMONY MELON', 'HARMONY LEMON', '...",frozenset({'HARMONY STRAWBERY'}),0.000113,1,1235.260000
1,1,"frozenset({'DELFI TOP TRIP/CHO 9GR', 'MOMOGI J...",frozenset({'MOMOGI CKL 6GR'}),0.000113,1,686.255556
2,2,"frozenset({'INDOMILK KID VNL 115ML', 'CLOUD 9 ...",frozenset({'CLOUD 9 CHOCO 15 4G'}),0.000113,1,571.879630


In [5]:
# Pastikan semua elemen dalam kolom antecedents dan consequents adalah frozenset
df["antecedents"] = df["antecedents"].apply(lambda x: frozenset(eval(x)) if isinstance(x, str) else x)
df["consequents"] = df["consequents"].apply(lambda x: frozenset(eval(x)) if isinstance(x, str) else x)

# Mengambil semua item unik
all_items = set()
for col in ["antecedents", "consequents"]:
    for items in df[col]:  # Loop agar tetap robust
        all_items.update(items)

all_items

{'CLOUD 9 CHOCO 15 4G',
 'CLOUD 9 VANILA 15 4G',
 'DELFI TOP TRIP/CHO 9GR',
 'HARMONY LEMON',
 'HARMONY MELON',
 'HARMONY ORANGE',
 'HARMONY STRAWBERY',
 'INDOMILK KID STR 115ML',
 'INDOMILK KID VNL 115ML',
 'MOMOGI CKL 6GR',
 'MOMOGI JGG BKR 6GR'}

In [6]:
len(all_items)

11

In [7]:
groceries = pd.read_excel('DATA2023.xlsx')
groceries

,TANGGAL,NO TRANSAKSI,NAMA BARANG,QTY
0,2023-01-01,2301011000001,FORVITA MARG 200GR,2
1,2023-01-01,2301011000002,SASA SANTAN KLPA 65ML,3
2,2023-01-01,2301011000003,CHEERS 1500ML GREEN,1
3,2023-01-01,2301011000004,SUN KARA 65ML,1
4,2023-01-01,2301011000004,OREO PIKACHU 165.6GR,1
...,...,...,...,...
162975,2023-12-31,2312311020112,KONIDIN 4'S,2
162976,2023-12-31,2312311020113,GOLDA CAPPUCINO 200ML,1
162977,2023-12-31,2312311020113,FLORIDINA ORANGE 360ML,1
162978,2023-12-31,2312311020114,WALLS PP TRICO/48 65298,1


In [8]:
groceries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   TANGGAL       162980 non-null  datetime64[ns]
 1   NO TRANSAKSI  162980 non-null  int64         
 2   NAMA BARANG   162980 non-null  object        
 3   QTY           162980 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 5.0+ MB


In [9]:
df_filtered = groceries[groceries['NAMA BARANG'].isin(all_items)]
df_filtered 

,TANGGAL,NO TRANSAKSI,NAMA BARANG,QTY
177,2023-01-01,2301011000075,INDOMILK KID STR 115ML,1
297,2023-01-01,2301011000108,MOMOGI CKL 6GR,2
324,2023-01-01,2301011000119,HARMONY ORANGE,1
325,2023-01-01,2301011000119,HARMONY LEMON,1
445,2023-01-01,2301011000157,HARMONY ORANGE,1
...,...,...,...,...
162382,2023-12-31,2312311010014,INDOMILK KID STR 115ML,1
162425,2023-12-31,2312311010032,INDOMILK KID STR 115ML,2
162562,2023-12-31,2312311010077,HARMONY STRAWBERY,4
162564,2023-12-31,2312311010077,HARMONY LEMON,2


In [10]:
# Agregasi jumlah QTY per tanggal dan nama barang
df_grouped = df_filtered.groupby(['TANGGAL', 'NAMA BARANG'], as_index=False)['QTY'].sum()

tanggal_range = pd.date_range(start='2023-01-01', end='2023-12-31')
all_items = df_filtered['NAMA BARANG'].unique()

# Buat kombinasi semua tanggal dan item
multi_index = pd.MultiIndex.from_product([tanggal_range, all_items], names=['TANGGAL', 'NAMA BARANG'])
all_combinations = pd.DataFrame(index=multi_index).reset_index()

# Gabungkan dengan hasil groupby
df_joined = pd.merge(all_combinations, df_grouped, on=['TANGGAL', 'NAMA BARANG'], how='left')

df_joined['QTY'] = df_joined['QTY'].fillna(0)
fig = px.line(df_joined, x='TANGGAL', y='QTY', color='NAMA BARANG',
              title='Jumlah Penjualan Harian per Item')
fig.update_layout(legend_title_text='Klik untuk tampilkan/sembunyikan item')
fig.show()

In [11]:
df_joined 

,TANGGAL,NAMA BARANG,QTY
0,2023-01-01,INDOMILK KID STR 115ML,1.0
1,2023-01-01,MOMOGI CKL 6GR,2.0
2,2023-01-01,HARMONY ORANGE,2.0
3,2023-01-01,HARMONY LEMON,1.0
4,2023-01-01,MOMOGI JGG BKR 6GR,0.0
...,...,...,...
3280,2023-12-31,MOMOGI JGG BKR 6GR,0.0
3281,2023-12-31,HARMONY STRAWBERY,4.0
3282,2023-12-31,DELFI TOP TRIP/CHO 9GR,0.0
3283,2023-12-31,HARMONY MELON,4.0


In [12]:
df_pivot = df_joined.pivot(index='TANGGAL', columns='NAMA BARANG', values='QTY')
df_pivot.columns.name = None  # hilangkan nama kolom atas
df_pivot = df_pivot.reset_index() 
df_pivot

,TANGGAL,DELFI TOP TRIP/CHO 9GR,HARMONY LEMON,HARMONY MELON,HARMONY ORANGE,HARMONY STRAWBERY,INDOMILK KID STR 115ML,INDOMILK KID VNL 115ML,MOMOGI CKL 6GR,MOMOGI JGG BKR 6GR
0,2023-01-01,0.0,1.0,0.0,2.0,0.0,1.0,0.0,2.0,0.0
1,2023-01-02,0.0,0.0,0.0,0.0,0.0,3.0,0.0,5.0,3.0
2,2023-01-03,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
3,2023-01-04,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0
4,2023-01-05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
360,2023-12-27,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
361,2023-12-28,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
362,2023-12-29,0.0,2.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0
363,2023-12-30,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0


In [13]:
for col in df_pivot.columns:
    if col != 'TANGGAL':
        result = adfuller(df_pivot[col].diff().dropna())
        print(f'{col}')
        print(f'ADF Statistic: {result[0]}')
        print(f'p-value: {result[1]}')
        print('\n---------------------\n')

DELFI TOP TRIP/CHO 9GR
ADF Statistic: -9.57340883902092
p-value: 2.2565010014339717e-16

---------------------

HARMONY LEMON
ADF Statistic: -9.020883053757704
p-value: 5.77982613666503e-15

---------------------

HARMONY MELON
ADF Statistic: -8.2511054682455
p-value: 5.388086192207207e-13

---------------------

HARMONY ORANGE
ADF Statistic: -10.230816924698905
p-value: 5.034529552712945e-18

---------------------

HARMONY STRAWBERY
ADF Statistic: -8.472423952726194
p-value: 1.4654562345319693e-13

---------------------

INDOMILK KID STR 115ML
ADF Statistic: -8.925148312193192
p-value: 1.016043446993352e-14

---------------------

INDOMILK KID VNL 115ML
ADF Statistic: -10.118340480424537
p-value: 9.591163469520532e-18

---------------------

MOMOGI CKL 6GR
ADF Statistic: -8.183778935621985
p-value: 8.000487929414566e-13

---------------------

MOMOGI JGG BKR 6GR
ADF Statistic: -7.555265701746692
p-value: 3.1115929214143497e-11

---------------------



In [14]:
from itertools import permutations
from statsmodels.tsa.stattools import grangercausalitytests

# Simpan hasil signifikan
significant_results = []

# Pilih hanya kolom item
item_cols = [col for col in df_pivot.columns if col != 'TANGGAL']

# Uji Granger untuk semua pasangan (x causes y)
for x, y in permutations(item_cols, 2):
    print(f'\n{x} causes {y}?')
    print('---------------------------')
    try:
        granger_result = grangercausalitytests(df_pivot[[y, x]], maxlag=5, verbose=False)
        for lag in range(1, 6):
            p_value = granger_result[lag][0]['ssr_ftest'][1]  # ambil p-value
            if p_value < 0.05:
                print(f'Significant at lag {lag} (p-value = {p_value:.4f})')
                significant_results.append({
                    'cause': x,
                    'effect': y,
                    'lag': lag,
                    'p_value': p_value
                })
                break  
            else:
                print(f'Not significant at lag {lag} (p-value = {p_value:.4f})')
    except Exception as e:
        print(f'Error: {e}')



DELFI TOP TRIP/CHO 9GR causes HARMONY LEMON?
---------------------------
Not significant at lag 1 (p-value = 0.2608)
Not significant at lag 2 (p-value = 0.5319)
Not significant at lag 3 (p-value = 0.7247)
Not significant at lag 4 (p-value = 0.8594)
Not significant at lag 5 (p-value = 0.9266)

DELFI TOP TRIP/CHO 9GR causes HARMONY MELON?
---------------------------
Not significant at lag 1 (p-value = 0.9278)
Not significant at lag 2 (p-value = 0.8724)
Not significant at lag 3 (p-value = 0.7301)
Not significant at lag 4 (p-value = 0.8386)
Not significant at lag 5 (p-value = 0.7240)

DELFI TOP TRIP/CHO 9GR causes HARMONY ORANGE?
---------------------------
Not significant at lag 1 (p-value = 0.0641)
Not significant at lag 2 (p-value = 0.1754)
Not significant at lag 3 (p-value = 0.2514)
Not significant at lag 4 (p-value = 0.3963)
Not significant at lag 5 (p-value = 0.5323)

DELFI TOP TRIP/CHO 9GR causes HARMONY STRAWBERY?
---------------------------
Not significant at lag 1 (p-value = 0.1

In [15]:
# Agregasi jumlah QTY per tanggal dan nama barang
df_grouped = df_filtered.groupby(['TANGGAL', 'NAMA BARANG'], as_index=False)['QTY'].sum()

tanggal_range = pd.date_range(start='2023-01-01', end='2023-12-31')
all_items = df_filtered['NAMA BARANG'].unique()

# Buat kombinasi semua tanggal dan item
multi_index = pd.MultiIndex.from_product([tanggal_range, all_items], names=['TANGGAL', 'NAMA BARANG'])
all_combinations = pd.DataFrame(index=multi_index).reset_index()

# Gabungkan dengan hasil groupby
df_joined = pd.merge(all_combinations, df_grouped, on=['TANGGAL', 'NAMA BARANG'], how='left')
df_joined['QTY'] = df_joined['QTY'].fillna(0)

# Pastikan kolom TANGGAL dalam format datetime
df_joined['TANGGAL'] = pd.to_datetime(df_joined['TANGGAL'])

# Agregasi mingguan
df_weekly = df_joined.groupby([
    pd.Grouper(key='TANGGAL', freq='W-MON'),  # Grup per minggu, mulai dari Senin
    'NAMA BARANG'
], as_index=False)['QTY'].sum()

# Visualisasi
fig = px.line(df_weekly, x='TANGGAL', y='QTY', color='NAMA BARANG',
              title='Jumlah Penjualan Mingguan per Item')
fig.update_layout(legend_title_text='Klik untuk tampilkan/sembunyikan item')
fig.show()


In [16]:
df_weekly

,TANGGAL,NAMA BARANG,QTY
0,2023-01-02,DELFI TOP TRIP/CHO 9GR,0.0
1,2023-01-02,HARMONY LEMON,1.0
2,2023-01-02,HARMONY MELON,0.0
3,2023-01-02,HARMONY ORANGE,2.0
4,2023-01-02,HARMONY STRAWBERY,0.0
...,...,...,...
472,2024-01-01,HARMONY STRAWBERY,4.0
473,2024-01-01,INDOMILK KID STR 115ML,8.0
474,2024-01-01,INDOMILK KID VNL 115ML,15.0
475,2024-01-01,MOMOGI CKL 6GR,0.0


In [17]:
df_weekly = df_weekly.pivot(index='TANGGAL', columns='NAMA BARANG', values='QTY')
df_weekly.columns.name = None  # hilangkan nama kolom atas
df_weekly = df_weekly.reset_index() 
df_weekly

,TANGGAL,DELFI TOP TRIP/CHO 9GR,HARMONY LEMON,HARMONY MELON,HARMONY ORANGE,HARMONY STRAWBERY,INDOMILK KID STR 115ML,INDOMILK KID VNL 115ML,MOMOGI CKL 6GR,MOMOGI JGG BKR 6GR
0,2023-01-02,0.0,1.0,0.0,2.0,0.0,4.0,0.0,7.0,3.0
1,2023-01-09,0.0,2.0,0.0,1.0,1.0,9.0,0.0,5.0,6.0
2,2023-01-16,4.0,3.0,0.0,0.0,1.0,4.0,0.0,8.0,5.0
3,2023-01-23,6.0,0.0,0.0,0.0,2.0,5.0,0.0,0.0,4.0
4,2023-01-30,1.0,0.0,0.0,0.0,1.0,9.0,0.0,2.0,7.0
5,2023-02-06,7.0,1.0,0.0,5.0,1.0,8.0,0.0,18.0,12.0
6,2023-02-13,3.0,0.0,0.0,0.0,0.0,11.0,0.0,11.0,12.0
7,2023-02-20,4.0,0.0,0.0,0.0,0.0,4.0,0.0,15.0,25.0
8,2023-02-27,4.0,0.0,0.0,0.0,0.0,7.0,0.0,12.0,11.0
9,2023-03-06,0.0,0.0,0.0,0.0,0.0,6.0,0.0,6.0,19.0


In [18]:
for col in df_weekly.columns:
    if col != 'TANGGAL':
        result = adfuller(df_weekly[col].diff().dropna())
        print(f'{col}')
        print(f'ADF Statistic: {result[0]}')
        print(f'p-value: {result[1]}')
        print('\n---------------------\n')

DELFI TOP TRIP/CHO 9GR
ADF Statistic: -4.579254319066549
p-value: 0.0001410330877995549

---------------------

HARMONY LEMON
ADF Statistic: -4.273323201260498
p-value: 0.0004939116103999298

---------------------

HARMONY MELON
ADF Statistic: -3.719904085389442
p-value: 0.003841738282517941

---------------------

HARMONY ORANGE
ADF Statistic: -4.966275858040494
p-value: 2.5868966819984078e-05

---------------------

HARMONY STRAWBERY
ADF Statistic: -4.193930863230703
p-value: 0.0006746614466328427

---------------------

INDOMILK KID STR 115ML
ADF Statistic: -3.88193375839732
p-value: 0.002171901805831974

---------------------

INDOMILK KID VNL 115ML
ADF Statistic: -9.859184838239115
p-value: 4.278264066069158e-17

---------------------

MOMOGI CKL 6GR
ADF Statistic: -7.491793292136388
p-value: 4.485414954855895e-11

---------------------

MOMOGI JGG BKR 6GR
ADF Statistic: -6.214105238764772
p-value: 5.4182453341761784e-08

---------------------



In [19]:
from itertools import permutations
from statsmodels.tsa.stattools import grangercausalitytests

# Simpan hasil signifikan
significant_results = []

# Pilih hanya kolom item
item_cols = [col for col in df_weekly.columns if col != 'TANGGAL']

# Uji Granger untuk semua pasangan (x causes y)
for x, y in permutations(item_cols, 2):
    print(f'\n{x} causes {y}?')
    print('---------------------------')
    try:
        granger_result = grangercausalitytests(df_weekly[[y, x]], maxlag=5, verbose=False)
        for lag in range(1, 6):
            p_value = granger_result[lag][0]['ssr_ftest'][1]  # ambil p-value
            if p_value < 0.05:
                print(f'Significant at lag {lag} (p-value = {p_value:.4f})')
                significant_results.append({
                    'cause': x,
                    'effect': y,
                    'lag': lag,
                    'p_value': p_value
                })
                break  
            else:
                print(f'Not significant at lag {lag} (p-value = {p_value:.4f})')
    except Exception as e:
        print(f'Error: {e}')



DELFI TOP TRIP/CHO 9GR causes HARMONY LEMON?
---------------------------
Not significant at lag 1 (p-value = 0.0909)
Not significant at lag 2 (p-value = 0.1503)
Not significant at lag 3 (p-value = 0.3360)
Not significant at lag 4 (p-value = 0.2923)
Not significant at lag 5 (p-value = 0.4305)

DELFI TOP TRIP/CHO 9GR causes HARMONY MELON?
---------------------------


Not significant at lag 1 (p-value = 0.2686)
Not significant at lag 2 (p-value = 0.3828)
Not significant at lag 3 (p-value = 0.4092)
Not significant at lag 4 (p-value = 0.5795)
Not significant at lag 5 (p-value = 0.6803)

DELFI TOP TRIP/CHO 9GR causes HARMONY ORANGE?
---------------------------
Not significant at lag 1 (p-value = 0.8536)
Not significant at lag 2 (p-value = 0.1174)
Not significant at lag 3 (p-value = 0.1653)
Not significant at lag 4 (p-value = 0.5047)
Not significant at lag 5 (p-value = 0.6037)

DELFI TOP TRIP/CHO 9GR causes HARMONY STRAWBERY?
---------------------------
Not significant at lag 1 (p-value = 0.6661)
Not significant at lag 2 (p-value = 0.8564)
Not significant at lag 3 (p-value = 0.9761)
Not significant at lag 4 (p-value = 0.9602)
Not significant at lag 5 (p-value = 0.9723)

DELFI TOP TRIP/CHO 9GR causes INDOMILK KID STR 115ML?
---------------------------
Significant at lag 1 (p-value = 0.0391)

DELFI TOP TRIP/CHO 9GR causes INDOMILK KID VNL 115ML?
---------

In [20]:
# Pastikan TANGGAL adalah index dan kolom item jadi variabel
df_weekly = df_weekly.fillna(0)

# Splitting 80:20
n_total = len(df_weekly)
n_train = int(n_total * 0.8)  # 80% untuk training

train_df = df_weekly.iloc[:n_train]
test_df = df_weekly.iloc[n_train:]

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

# Pilih lag optimal (optional tapi recommended)
model = VAR(train_df.diff().dropna())
sorted_order = model.select_order(maxlags=4)  
print(sorted_order.summary())

# Misalnya kita ambil lag terbaik = 4 (ubah sesuai hasil terbaik dari summary di atas)
best_lag = 4
var_model = VARMAX(train_df, order=(best_lag, 0), enforce_stationarity=True)
fitted_model = var_model.fit(disp=False)
print(fitted_model.summary())

# Forecast sebanyak panjang test
n_forecast = len(test_df)
pred = fitted_model.get_prediction(start=len(train_df), end=len(train_df) + n_forecast - 1)
predictions = pred.predicted_mean
predictions.columns = [f"{col}_predicted" for col in predictions.columns]

# Gabungkan prediksi dan aktual
test_vs_pred = pd.concat([test_df.reset_index(drop=True), predictions.reset_index(drop=True)], axis=1)

# Visualisasi seluruh item
test_vs_pred.plot(figsize=(15, 6), title="Prediksi vs Aktual - Semua Item")
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

# Tampilkan hasil prediksi dalam DataFrame
predicted_df = pd.concat([test_df.reset_index(drop=True), predictions.reset_index(drop=True)], axis=1)

# Menampilkan hasil prediksi dalam DataFrame
print(predicted_df)

Train shape: (42, 10)
Test shape: (11, 10)


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [ ]:
from sklearn.metrics import mean_absolute_error

# Hitung dan tampilkan RMSE, MAE, dan MAPE per item
for col in test_df.columns:
    actual = test_df[col]
    predicted = predictions[f"{col}_predicted"]

    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)

    # Untuk MAPE, hindari pembagian dengan nol
    mape = np.mean(np.abs((actual - predicted) / actual.replace(0, np.nan))) * 100

    print(f"Item: {col}")
    print(f"  Mean: {actual.mean():.2f}")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAE : {mae:.2f}")
    print(f"  MAPE: {mape:.2f}%\n")


Item: DELFI TOP TRIP/CHO 9GR
  Mean: 3.09
  RMSE: 2.94
  MAE : 2.61
  MAPE: 117.66%

Item: INDOMILK KID STR 115ML
  Mean: 15.55
  RMSE: 11.09
  MAE : 8.02
  MAPE: 42.99%

Item: INDOMILK KID VNL 115ML
  Mean: 9.18
  RMSE: 9.12
  MAE : 7.68
  MAPE: 76.40%

Item: MOMOGI CKL 6GR
  Mean: 3.27
  RMSE: 4.42
  MAE : 3.77
  MAPE: 154.08%

Item: MOMOGI JGG BKR 6GR
  Mean: 3.09
  RMSE: 7.00
  MAE : 6.06
  MAPE: 461.45%

